In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow_hub as hub
import tensorflow_text as text
df_stock = pd.read_csv("maruti_suzuki/MARUTI.NS.csv",
                 parse_dates = ["Date"],
                 index_col = ["Date"])
df_posts = pd.read_excel("maruti_suzuki/maruti_suzuki_final_posts.xlsx")
# removing the unnecessary columns
df_posts.drop(["Unnamed: 0"], axis=1,inplace=True)
# removing spam posts
df_posts = df_posts[df_posts.Spam==0.0]
df_posts.drop(["Spam"],axis=1,inplace=True)
# sliding a window of 7 days and adding all the TIs
from stock_helper import prepare_data
x,y = prepare_data(df_stock)
final_x = x[np.datetime64("2021-11-13"):]
final_y = y[np.datetime64("2021-11-13"):]
# reversing the posts data
df_posts = df_posts[::-1]
final_posts = df_posts[7:]
# loading the sentiment analysis model
sent_model = tf.keras.models.load_model("final_bert")
# removing duplicates from the data
final_posts.drop_duplicates(subset=['Messages'])
# calculating the sentiments score
sentiments = []
prev = np.datetime64("2015-11-12 21:31:26")
for i in final_y.index:
    total=0
    cnt=0
    for j in final_posts.itertuples():
        _,msg,time = j
        if np.datetime64(time)<np.datetime64(i) and np.datetime64(time)>prev:
            total += tf.squeeze(sent_model.predict([msg])).numpy()
            cnt+=1
    prev = np.datetime64(i)
    if(cnt==0):
        sentiments.append(0)
    else:
        sentiments.append(total/cnt)
# getting indices where sentiments score is 0
zero_index = []
for i,j in enumerate(sentiments):
    if(j==0):
        zero_index.append(i)
# removing all the zero values indices
sentiments = np.delete(sentiments,zero_index)
final_x_zeros = final_x.copy()
final_y_zeros = final_y.copy()
final_y_zeros = final_y_zeros.to_frame()
final_x_zeros['removal_assist'] = np.arange(0,len(final_x),1)
final_y_zeros['removal_assist'] = np.arange(0,len(final_x),1)
final_y_zeros = final_y_zeros[final_y_zeros.removal_assist.isin(zero_index)==False]
final_x_zeros = final_x_zeros[final_x_zeros.removal_assist.isin(zero_index)==False]
# removing the added helper column
final_x_zeros.drop(["removal_assist"], axis=1,inplace=True)
final_y_zeros.drop(["removal_assist"], axis=1, inplace=True)

In [13]:
tf.random.set_seed(42)
inputs = tf.keras.Input(shape=(71,1), name="inputs")
x = tf.keras.layers.Conv1D(64, 3, activation="relu", padding="causal")(inputs)
x = tf.keras.layers.MaxPooling1D()(x)
x = tf.keras.layers.Conv1D(64, 3, activation="relu", padding="causal")(x)
x = tf.keras.layers.MaxPooling1D()(x)
x = tf.keras.layers.GlobalMaxPooling1D()(x)
outputs = tf.keras.layers.Dense(1)(x)
model = tf.keras.models.Model(inputs, outputs)
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 71, 1)]           0         
                                                                 
 conv1d_6 (Conv1D)           (None, 71, 64)            256       
                                                                 
 max_pooling1d_6 (MaxPooling  (None, 35, 64)           0         
 1D)                                                             
                                                                 
 conv1d_7 (Conv1D)           (None, 35, 64)            12352     
                                                                 
 max_pooling1d_7 (MaxPooling  (None, 17, 64)           0         
 1D)                                                             
                                                                 
 global_max_pooling1d_3 (Glo  (None, 64)               0   

In [18]:
model_nbeats = tf.keras.models.load_model("nbeats_maruti suzuki")
model_nbeats.evaluate(final_x_zeros[50:],final_y_zeros[50:])

1/1 [==============================] - 1s 693ms/step - loss: 164.8210 - mae: 164.8210 - mse: 55453.7305


[164.8209686279297, 164.8209686279297, 55453.73046875]

In [16]:
model.compile(loss="mae",optimizer = tf.keras.optimizers.Adam(), metrics=["mae","mse"])
history = model.fit(final_x_zeros[:50],final_y_zeros[:50],
                    epochs = 1000,
                    validation_data = (final_x_zeros[50:], final_y_zeros[50:]),
                   callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", 
                                                        patience=200, 
                                                        restore_best_weights=True),
                      tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", 
                                                           patience=100, 
                                                           verbose=1)])

Epoch 1/1000
2/2 [==============================] - 1s 186ms/step - loss: 8310.7383 - mae: 8310.7383 - mse: 69434632.0000 - val_loss: 7893.9048 - val_mae: 7893.9048 - val_mse: 62330960.0000 - lr: 0.0010
Epoch 2/1000
2/2 [==============================] - 0s 34ms/step - loss: 6665.4575 - mae: 6665.4575 - mse: 44742888.0000 - val_loss: 6190.3945 - val_mae: 6190.3945 - val_mse: 38337744.0000 - lr: 0.0010
Epoch 3/1000
2/2 [==============================] - 0s 33ms/step - loss: 5133.6436 - mae: 5133.6436 - mse: 26671494.0000 - val_loss: 4578.5156 - val_mae: 4578.5156 - val_mse: 20980694.0000 - lr: 0.0010
Epoch 4/1000
2/2 [==============================] - 0s 32ms/step - loss: 3662.8525 - mae: 3662.8525 - mse: 13684516.0000 - val_loss: 3000.0515 - val_mae: 3000.0515 - val_mse: 9020799.0000 - lr: 0.0010
Epoch 5/1000
2/2 [==============================] - 0s 34ms/step - loss: 2217.4780 - mae: 2217.4780 - mse: 5216051.0000 - val_loss: 1435.2510 - val_mae: 1435.2510 - val_mse: 2083841.0000 - lr:

Epoch 43/1000
2/2 [==============================] - 0s 29ms/step - loss: 260.6480 - mae: 260.6480 - mse: 102668.5781 - val_loss: 223.0188 - val_mae: 223.0188 - val_mse: 77988.8672 - lr: 0.0010
Epoch 44/1000
2/2 [==============================] - 0s 41ms/step - loss: 259.2742 - mae: 259.2742 - mse: 102856.2812 - val_loss: 306.6174 - val_mae: 306.6174 - val_mse: 121985.3203 - lr: 0.0010
Epoch 45/1000
2/2 [==============================] - 0s 29ms/step - loss: 258.0435 - mae: 258.0435 - mse: 104514.8828 - val_loss: 385.2404 - val_mae: 385.2404 - val_mse: 176111.7656 - lr: 0.0010
Epoch 46/1000
2/2 [==============================] - 0s 27ms/step - loss: 260.8630 - mae: 260.8630 - mse: 110889.4531 - val_loss: 297.4549 - val_mae: 297.4549 - val_mse: 116616.8203 - lr: 0.0010
Epoch 47/1000
2/2 [==============================] - 0s 28ms/step - loss: 257.4402 - mae: 257.4402 - mse: 102654.6562 - val_loss: 198.2677 - val_mae: 198.2677 - val_mse: 67879.8828 - lr: 0.0010
Epoch 48/1000
2/2 [========

2/2 [==============================] - 0s 29ms/step - loss: 229.5685 - mae: 229.5685 - mse: 84792.2812 - val_loss: 226.3483 - val_mae: 226.3483 - val_mse: 80064.7734 - lr: 0.0010
Epoch 86/1000
2/2 [==============================] - 0s 30ms/step - loss: 229.2029 - mae: 229.2029 - mse: 84431.5625 - val_loss: 248.3190 - val_mae: 248.3190 - val_mse: 90396.0000 - lr: 0.0010
Epoch 87/1000
2/2 [==============================] - 0s 28ms/step - loss: 230.3970 - mae: 230.3970 - mse: 85310.0781 - val_loss: 255.3344 - val_mae: 255.3344 - val_mse: 93918.2109 - lr: 0.0010
Epoch 88/1000
2/2 [==============================] - 0s 29ms/step - loss: 230.5290 - mae: 230.5290 - mse: 85633.5469 - val_loss: 234.0661 - val_mae: 234.0661 - val_mse: 83620.2344 - lr: 0.0010
Epoch 89/1000
2/2 [==============================] - 0s 29ms/step - loss: 227.2812 - mae: 227.2812 - mse: 82623.2891 - val_loss: 164.3902 - val_mae: 164.3902 - val_mse: 56117.0000 - lr: 0.0010
Epoch 90/1000
2/2 [==============================

2/2 [==============================] - 0s 27ms/step - loss: 220.5229 - mae: 220.5229 - mse: 76893.9375 - val_loss: 170.2319 - val_mae: 170.2319 - val_mse: 57676.3867 - lr: 1.0000e-04
Epoch 127/1000
2/2 [==============================] - 0s 29ms/step - loss: 219.9949 - mae: 219.9949 - mse: 76066.6953 - val_loss: 179.0902 - val_mae: 179.0902 - val_mse: 60735.4883 - lr: 1.0000e-04
Epoch 128/1000
2/2 [==============================] - 0s 30ms/step - loss: 220.0203 - mae: 220.0203 - mse: 75975.3281 - val_loss: 189.6987 - val_mae: 189.6987 - val_mse: 64607.3320 - lr: 1.0000e-04
Epoch 129/1000
2/2 [==============================] - 0s 28ms/step - loss: 219.9661 - mae: 219.9661 - mse: 76008.9375 - val_loss: 199.5429 - val_mae: 199.5429 - val_mse: 68397.6094 - lr: 1.0000e-04
Epoch 130/1000
2/2 [==============================] - 0s 28ms/step - loss: 219.8948 - mae: 219.8948 - mse: 76117.0938 - val_loss: 210.7256 - val_mae: 210.7256 - val_mse: 72945.7188 - lr: 1.0000e-04
Epoch 131/1000
2/2 [=====

2/2 [==============================] - 0s 28ms/step - loss: 219.2518 - mae: 219.2518 - mse: 75737.0781 - val_loss: 203.2083 - val_mae: 203.2083 - val_mse: 69812.2188 - lr: 1.0000e-04
Epoch 168/1000
2/2 [==============================] - 0s 28ms/step - loss: 219.1161 - mae: 219.1161 - mse: 75633.4609 - val_loss: 198.2709 - val_mae: 198.2709 - val_mse: 67867.6328 - lr: 1.0000e-04
Epoch 169/1000
2/2 [==============================] - 0s 29ms/step - loss: 218.9837 - mae: 218.9837 - mse: 75444.1016 - val_loss: 192.0581 - val_mae: 192.0581 - val_mse: 65477.2070 - lr: 1.0000e-04
Epoch 170/1000
2/2 [==============================] - 0s 30ms/step - loss: 219.0130 - mae: 219.0130 - mse: 75365.3516 - val_loss: 184.5970 - val_mae: 184.5970 - val_mse: 62705.1211 - lr: 1.0000e-04
Epoch 171/1000
2/2 [==============================] - 0s 27ms/step - loss: 219.3523 - mae: 219.3523 - mse: 75234.5938 - val_loss: 178.1090 - val_mae: 178.1090 - val_mse: 60382.6562 - lr: 1.0000e-04
Epoch 172/1000
2/2 [=====

2/2 [==============================] - 0s 28ms/step - loss: 217.6738 - mae: 217.6738 - mse: 74716.2031 - val_loss: 195.7524 - val_mae: 195.7524 - val_mse: 66908.8359 - lr: 1.0000e-04
Epoch 209/1000
2/2 [==============================] - 0s 28ms/step - loss: 217.4043 - mae: 217.4043 - mse: 74492.8984 - val_loss: 188.9634 - val_mae: 188.9634 - val_mse: 64329.9805 - lr: 1.0000e-04
Epoch 210/1000
2/2 [==============================] - 0s 28ms/step - loss: 217.3029 - mae: 217.3029 - mse: 74092.4766 - val_loss: 181.0332 - val_mae: 181.0332 - val_mse: 61427.1445 - lr: 1.0000e-04
Epoch 211/1000
2/2 [==============================] - 0s 28ms/step - loss: 217.1665 - mae: 217.1665 - mse: 73985.3516 - val_loss: 172.3027 - val_mae: 172.3027 - val_mse: 58373.6836 - lr: 1.0000e-04
Epoch 212/1000
2/2 [==============================] - 0s 28ms/step - loss: 217.5124 - mae: 217.5124 - mse: 74338.5781 - val_loss: 160.1774 - val_mae: 160.1774 - val_mse: 54244.9414 - lr: 1.0000e-04
Epoch 213/1000
2/2 [=====

In [17]:
model.evaluate(final_x_zeros[50:],final_y_zeros[50:])

1/1 [==============================] - 0s 27ms/step - loss: 113.8574 - mae: 113.8574 - mse: 27533.9824


[113.857421875, 113.857421875, 27533.982421875]

In [19]:
model.save("conv1d_maruti suzuki")

INFO:tensorflow:Assets written to: conv1d_maruti suzuki\assets
